In [ ]:
#!pip install noisereduce

### DAIC-WOZ 음성 데이터 전처리 
- Participant 음성만 남김
- 잡음 제거
- 1분 이내로 끊음

In [1]:
import pandas as pd
from pydub import AudioSegment
import noisereduce as nr
import numpy as np


c:\Users\Jws\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
c:\Users\Jws\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

def process_audio(start_id, end_id, directory):

    for file_id in range(start_id, end_id + 1):
        audio_path = f"{directory}/{file_id}_AUDIO.wav"
        transcript_path = f"{directory}/{file_id}_TRANSCRIPT.csv"

        try:
        
            # TRANSCRIPT.csv 파일에서 Participant 의 대사만 추출
            transcript = pd.read_csv(transcript_path, sep="\t")
            participant_transcript = transcript[transcript['speaker'] == 'Participant']
            
            # 오디오 파일 로드
            audio = AudioSegment.from_wav(audio_path)
            
            # 최종 오디오를 생성할 리스트
            final_audio = AudioSegment.silent(duration=0)
            
            # 라벨 
            n = 1

            # 대사별 오디오 처리 및 결합
            for index, row in participant_transcript.iterrows():
                start_time = int(row['start_time'] * 1000)  # 시작 시간을 밀리세컨드로 변환
                end_time = int(row['stop_time'] * 1000)  # 종료 시간을 밀리세컨드로 변환
                
                # 잘라낸 오디오
                segment = audio[start_time:end_time]
                
                # 잡음 제거
                segment_np = np.array(segment.get_array_of_samples())
                reduced_noise = nr.reduce_noise(y=segment_np, sr=segment.frame_rate)
                
                # pydub 오디오로 변환
                clean_segment = AudioSegment(
                    data=reduced_noise.tobytes(),
                    sample_width=segment.sample_width,
                    frame_rate=segment.frame_rate,
                    channels=segment.channels
                )
                
                # 결합 전 길이 확인 및 파일 분할
                if len(final_audio) + len(clean_segment) > 60000:  # 1분 초과 시 새 파일 시작
                    final_audio.export(f"../data/reduce/{file_id}_processed_{n}.wav", format="wav")
                    final_audio = clean_segment
                    n += 1
                else:
                    final_audio += clean_segment
            
            # 마지막 파일 저장
            if len(final_audio) > 0:
                final_audio.export(f"../data/reduce/{file_id}_processed_final.wav", format="wav")
        except Exception as e:
            print(f"Error processing file {file_id}: {e}")
            


In [3]:
# 함수 실행
process_audio(300, 492, '../data/out')


c:\Users\Jws\AppData\Local\Programs\Python\Python312\Lib\site-packages\noisereduce\spectralgate\nonstationary.py:71: RuntimeWarning: invalid value encountered in divide
  sig_mult_above_thresh = (abs_sig_stft - sig_stft_smooth) / sig_stft_smooth
c:\Users\Jws\AppData\Local\Programs\Python\Python312\Lib\site-packages\noisereduce\spectralgate\base.py:224: RuntimeWarning: invalid value encountered in cast
  return filtered_chunk.astype(self._dtype).flatten()
c:\Users\Jws\AppData\Local\Programs\Python\Python312\Lib\site-packages\noisereduce\spectralgate\nonstationary.py:71: RuntimeWarning: invalid value encountered in divide
  sig_mult_above_thresh = (abs_sig_stft - sig_stft_smooth) / sig_stft_smooth
c:\Users\Jws\AppData\Local\Programs\Python\Python312\Lib\site-packages\noisereduce\spectralgate\base.py:224: RuntimeWarning: invalid value encountered in cast
  return filtered_chunk.astype(self._dtype).flatten()
c:\Users\Jws\AppData\Local\Programs\Python\Python312\Lib\site-packages\noisereduce

#### 데이터셋 분할 저장
train,dev,test

In [3]:
import pandas as pd
from pydub import AudioSegment
import numpy as np
import noisereduce as nr



c:\Users\Jws\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
c:\Users\Jws\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:

def load_splits(directory):
    # 분할 정보 로드
    dev = pd.read_csv(f"{directory}/dev_split_Depression.csv")
    test = pd.read_csv(f"{directory}/full_test_split.csv")
    train = pd.read_csv(f"{directory}/train_split_Depression.csv")

    # 분할을 딕셔너리로 구성
    splits = {}
    splits.update(dict.fromkeys(dev['Participant_ID'], 'dev'))
    splits.update(dict.fromkeys(test['Participant_ID'], 'test'))
    splits.update(dict.fromkeys(train['Participant_ID'], 'train'))
    return splits

def process_audio(start_id, end_id, directory, splits):
    for file_id in range(start_id, end_id + 1):
        audio_path = f"{directory}/{file_id}_AUDIO.wav"
        transcript_path = f"{directory}/{file_id}_TRANSCRIPT.csv"

        try:
            # TRANSCRIPT.csv 파일에서 Participant 의 대사만 추출
            transcript = pd.read_csv(transcript_path, sep="\t")
            participant_transcript = transcript[transcript['speaker'] == 'Participant']

            # 오디오 파일 로드
            audio = AudioSegment.from_wav(audio_path)

            # 최종 오디오를 생성할 리스트
            final_audio = AudioSegment.silent(duration=0)

            # 라벨 초기화
            n = 1
            split_folder = splits.get(file_id, 'unknown')

            # 대사별 오디오 처리 및 결합
            for index, row in participant_transcript.iterrows():
                start_time = int(row['start_time'] * 1000)  # 시작 시간을 밀리세컨드로 변환
                end_time = int(row['stop_time'] * 1000)  # 종료 시간을 밀리세컨드로 변환
                
                # 잘라낸 오디오
                segment = audio[start_time:end_time]
                
                # 잡음 제거
                segment_np = np.array(segment.get_array_of_samples())
                reduced_noise = nr.reduce_noise(y=segment_np, sr=segment.frame_rate)
                
                # pydub 오디오로 변환
                clean_segment = AudioSegment(
                    data=reduced_noise.tobytes(),
                    sample_width=segment.sample_width,
                    frame_rate=segment.frame_rate,
                    channels=segment.channels
                )
                
                # 결합 전 길이 확인 및 파일 분할
                if len(final_audio) + len(clean_segment) > 60000:  # 1분 초과 시 새 파일 시작
                    final_audio.export(f"../data/reduce/{split_folder}/{file_id}_processed_{n}.wav", format="wav")
                    final_audio = clean_segment
                    n += 1
                else:
                    final_audio += clean_segment
            
            # 마지막 파일 저장
            if len(final_audio) > 0:
                final_audio.export(f"../data/reduce/{split_folder}/{file_id}_processed_final.wav", format="wav")
                print(f"{file_id}_final 저장완료")
        except Exception as e:
            print(f"Error processing file {file_id}: {e}")


In [ ]:

# 분할 정보 로드
splits = load_splits('../split')

# 함수 실행
process_audio(300, 492, '../data/out', splits)


#### librosa로 잡음 제거

In [4]:
import pandas as pd
from pydub import AudioSegment
import librosa
import numpy as np
import soundfile as sf
import os


c:\Users\Jws\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:

def reduce_noise_librosa(audio_segment):
    # Pydub 오디오를 Numpy 배열로 변환
    audio_np = np.array(audio_segment.get_array_of_samples())
    sr = audio_segment.frame_rate

    # Librosa를 이용해 오디오 파일 읽기
    y = np.float32(audio_np) / 2**15

    # 잡음이 있다고 판단되는 부분을 찾아 잡음 프로필을 생성
    noise_part = y[:int(sr*0.5)]  # 예를 들어, 처음 0.5초간의 오디오를 잡음으로 가정
    noise_profile = np.mean(librosa.feature.mfcc(y=noise_part, sr=sr, n_mfcc=20), axis=1)
    noise = np.tile(noise_profile, (len(y) // len(noise_profile), 1)).flatten()[:len(y)]

    # 원 오디오에서 잡음 프로필을 제거
    y_reduced = y - noise

    # 결과를 다시 Pydub 오디오로 변환
    reduced_np = np.int16(y_reduced * 2**15)
    clean_segment = AudioSegment(
        data=reduced_np.tobytes(),
        sample_width=audio_segment.sample_width,
        frame_rate=sr,
        channels=audio_segment.channels
    )
    
    return clean_segment

def process_audio(start_id, end_id, directory):
    for file_id in range(start_id, end_id + 1):
        audio_path = f"{directory}/{file_id}_AUDIO.wav"
        transcript_path = f"{directory}/{file_id}_TRANSCRIPT.csv"
        
        try:
            # TRANSCRIPT.csv 파일에서 Participant 의 대사만 추출
            transcript = pd.read_csv(transcript_path, sep='\t')
            participant_transcript = transcript[transcript['speaker'] == 'Participant']
            
            # 오디오 파일 로드
            audio = AudioSegment.from_wav(audio_path)
            
            # 최종 오디오를 생성할 리스트
            final_audio = AudioSegment.silent(duration=0)
            
            # 라벨링
            n = 1

            # 대사별 오디오 처리 및 결합
            for index, row in participant_transcript.iterrows():
                start_time = int(row['start_time'] * 1000)
                stop_time = int(row['stop_time'] * 1000)
                
                # 잘라낸 오디오
                segment = audio[start_time:stop_time]
                
                # 잡음 제거
                clean_segment = reduce_noise_librosa(segment)
                
                # 결합 전 길이 확인 및 파일 분할
                if len(final_audio) + len(clean_segment) > 60000:
                    final_audio.export(f"../data/librosa/{file_id}_processed_{n}.wav", format="wav")
                    final_audio = clean_segment
                    n += 1
                else:
                    final_audio += clean_segment
            
            # 마지막 파일 저장
            if len(final_audio) > 0:
                final_audio.export(f"../data/librosa/{file_id}_processed_final.wav", format="wav")
        
        except Exception as e:
            print(f"Error processing file {file_id}: {e}")


In [5]:

# 예제 사용 방법
process_audio(300, 492, '../data/out')


Error processing file 301: operands could not be broadcast together with shapes (19856,) (19840,) 
Error processing file 302: operands could not be broadcast together with shapes (45328,) (45320,) 
Error processing file 303: operands could not be broadcast together with shapes (15056,) (15040,) 
Error processing file 304: operands could not be broadcast together with shapes (7024,) (7020,) 
Error processing file 305: operands could not be broadcast together with shapes (240784,) (240780,) 
Error processing file 306: operands could not be broadcast together with shapes (185584,) (185580,) 
Error processing file 307: operands could not be broadcast together with shapes (193632,) (193620,) 
Error processing file 308: operands could not be broadcast together with shapes (9776,) (9760,) 
Error processing file 309: operands could not be broadcast together with shapes (21616,) (21600,) 
Error processing file 310: operands could not be broadcast together with shapes (4944,) (4940,) 
Error proc